In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [96]:

class multi_head(nn.Module):
    def __init__(self):
        super(multi_head, self).__init__()
        self.A = nn.ModuleList([nn.Linear(32, 1) for _ in range(2)])
        self.weight_init()

    def weight_init(self):
        for i in range(2):
            nn.init.xavier_normal_(self.A[i].weight)
            self.A[i].bias.data.fill_(0.0)
    
    def attn_summary(self, features):
        features_attn = []
        k = features[0].shape[0]
        for i in range(2):
            features_attn.append((self.A[i](features[i].squeeze())))
        features_attn = F.softmax(torch.cat(features_attn), dim=-1).unsqueeze(1)
        features = torch.cat(features)
        features = (features * features)
        
        # k = features[i].shape[0]
        print(k)
        features = torch.cat((features[:k],features[k:]),axis=1)
        
        print(features.shape)
        print(features)
        # features = features.reshape(-1,64)
        return features, features_attn

# transposed_tensor = input_tensor.transpose(0, 1)

# # Repeat each row of the transposed tensor
# output_tensor = transposed_tensor.unsqueeze(1).expand(-1, len(input_tensor), -1).reshape(-1, len(input_tensor[0]))

# print(output_tensor)


    def forward(self,x):
        # x1,x2,x3,x4 = x[:,:16], x[:,16:32], x[:,32:48], x[:,48:]
        x1,x2 = x[:,:32], x[:,32:]

        # results = self.attn_summary([x1,x2,x3,x4])
        results = self.attn_summary([x1,x2])

        return results
        
    

In [98]:

class Residual(nn.Module):
    def __init__(self):
        super(Residual,self).__init__()
        self.mlp1 = nn.Linear(64,64)
        self.mlp2 = nn.Linear(64,32)
        self.mlp3 = nn.Linear(32,16)
        self.mlp4 = nn.Linear(16,32)
        self.mlp5 = nn.Linear(32,64)
        self.act = nn.Tanh()
    def forward(self, features):
        x1 = self.mlp1(features)
        x1 = self.act(x1)
        
        x1 = self.mlp2(features)
        x1 = self.act(x1)
        x1 = self.mlp3(features)
        x1 = self.act(x1)
        
        x1 = self.mlp4(features)
        x1 = self.act(x1)
        
        x1 = self.mlp5(features)
        x1 = x1 + features
        
        x1 = self.act(x1)
        return x1

In [99]:
a = Residual()

b = torch.FloatTensor([[i for i in range(64)] for j in range(5)])

aa = a(b)
# print(b.shape)
# print(aa.shape)
# aa

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x64 and 32x16)